In [1]:
from tensorflow.keras.preprocessing.image  import ImageDataGenerator

In [2]:
from tensorflow.keras.applications import MobileNetV2

In [3]:
from tensorflow.keras.layers import AveragePooling2D

In [4]:
from tensorflow.keras.layers import Dropout

In [5]:
from tensorflow.keras.layers import Flatten

In [6]:
from tensorflow.keras.layers import Dense

In [7]:
from tensorflow.keras.layers import Input 

In [8]:
from tensorflow.keras.models import Model

In [9]:
from tensorflow.keras.optimizers import Adam

In [10]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

In [11]:
from tensorflow.keras.preprocessing.image import img_to_array

In [12]:
from tensorflow.keras.preprocessing.image import load_img

In [13]:
from tensorflow.keras.utils import to_categorical

In [14]:
from sklearn.preprocessing import LabelBinarizer

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
from sklearn.metrics import classification_report

In [17]:
from imutils import paths

In [18]:
import matplotlib.pyplot as plt

In [19]:
import numpy as np

In [20]:
import os

In [21]:
INIT_LR=1e-4

In [22]:
EPOCHS =20

In [23]:
BS = 32

In [24]:
DIRECTORY=r"C:\Users\ADMIN\Downloads\face mask\Dataset"

In [25]:
CATEGORIES =["with_mask","without_mask"]

In [26]:
print("LETS LOAD IMAGES")

LETS LOAD IMAGES


In [27]:
data=[]

In [28]:
labels=[]

In [29]:
for category in CATEGORIES:
    path= os.path.join(DIRECTORY,category)
for img in os.listdir(path):
    img_path= os.path.join(path,img)
    image=load_img(img_path,target_size=(224,224))
    image=img_to_array(image)
    image = preprocess_input(image)

    data.append(image)
    labels.append(category)

In [30]:
lb=LabelBinarizer()
labels=lb.fit_transform(labels)
labels=to_categorical(labels)

In [31]:
data= np.array(data,dtype="float32")
labels= np.array(labels)

In [32]:
(trainX, testX,trainY,testY)=train_test_split(data,labels,
test_size=0.20 , stratify=labels,random_state=10)

 #Now we will construct training image generator 

In [33]:
aug=ImageDataGenerator(
rotation_range=20,
zoom_range=0.15,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.15,
horizontal_flip=True,
fill_mode="nearest")

In [34]:
baseModel = MobileNetV2(weights="imagenet", include_top=False,
 input_tensor=Input(shape=(224, 224,3)))

In [35]:
baseModel.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

                                                                 block_12_project_BN[0][0]        
__________________________________________________________________________________________________
block_13_expand (Conv2D)        (None, 14, 14, 576)  55296       block_12_add[0][0]               
__________________________________________________________________________________________________
block_13_expand_BN (BatchNormal (None, 14, 14, 576)  2304        block_13_expand[0][0]            
__________________________________________________________________________________________________
block_13_expand_relu (ReLU)     (None, 14, 14, 576)  0           block_13_expand_BN[0][0]         
__________________________________________________________________________________________________
block_13_pad (ZeroPadding2D)    (None, 15, 15, 576)  0           block_13_expand_relu[0][0]       
__________________________________________________________________________________________________
block_13_d

In [36]:
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)



In [37]:
model = Model(inputs=baseModel.input , outputs=headModel)

In [38]:
for layer in baseModel.layers:
    layer.trainable= False

In [ ]:
print("complie it ")
opt= Adam(learning_rate=INIT_LR,decay=INIT_LR/EPOCHS)
model.compile(loss="binary_crossentropy",optimizer=opt,
     metrics=["accuracy"])
print("[INFO] training head...")
H = model.fit(
 aug.flow(trainX, trainY, batch_size=BS),
 steps_per_epoch=len(trainX) // BS,
 validation_data=(testX, testY),
 validation_steps=len(testX) // BS,
 epochs=EPOCHS)

model.save(r"C:\Users\ADMIN\Downloads\face mask\mobilenetv2chetan")


In [42]:
y_true = [0, 1, 1, 1, 1]
>>> y_pred = [0, 0, 1, 1, 1]
>>> target_names = ['with_mask','without_mask']
>>> print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

   with_mask       0.50      1.00      0.67         1
without_mask       1.00      0.75      0.86         4

    accuracy                           0.80         5
   macro avg       0.75      0.88      0.76         5
weighted avg       0.90      0.80      0.82         5



In [43]:
print("saving the model")
model.save("mask.detect.model",save_format="h5")

saving the model


In [ ]:
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(plot)

NameError: name 'plot' is not defined